# Multi-Document Agents (V1)
이 가이드에서는 LlamaIndex 문서를 통해 다중 문서 에이전트를 설정하는 방법을 알아봅니다.

이는 다음과 같은 추가 기능을 갖춘 V0 다중 문서 에이전트의 확장입니다.
- 문서(도구) 검색 중 순위 재지정
- 에이전트가 계획을 세우는 데 사용할 수 있는 쿼리 계획 도구

우리는 다음 아키텍처를 사용하여 이를 수행합니다.
- 각 문서에 대해 "문서 에이전트"를 설정합니다. 각 문서 에이전트는 해당 문서 내에서 QA/요약을 수행할 수 있습니다.
- 이 문서 에이전트 집합에 대해 최상위 에이전트를 설정합니다. 도구 검색을 수행한 다음 도구 세트에 대해 CoT를 수행하여 질문에 답합니다.

## 데이터 설정 및 다운로드
이 섹션에서는 LlamaIndex 문서를 로드합니다.

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
domain = "docs.llamaindex.ai"
docs_url = "https://docs.llamaindex.ai/en/latest/"
!wget -e robots=off --recursive --no-clobber --page-requisites --html-extension --convert-links --restrict-file-names=windows --domains {domain} --no-parent {docs_url}

In [8]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/heewungsong/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

#### Load HTML Files

In [1]:
from pathlib import Path

all_files_gen = Path("./docs.llamaindex.ai/").rglob("*")
all_files = [f.resolve() for f in all_files_gen]

all_html_files = [f for f in all_files if f.suffix.lower() == ".html"]

print(len(all_html_files))

1197


In [2]:
all_html_files[:5]

[PosixPath('/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/Agents/docs.llamaindex.ai/en/latest/index.html'),
 PosixPath('/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/Agents/docs.llamaindex.ai/en/latest/understanding/index.html'),
 PosixPath('/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/Agents/docs.llamaindex.ai/en/latest/understanding/using_llms/using_llms/index.html'),
 PosixPath('/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/Agents/docs.llamaindex.ai/en/latest/understanding/using_llms/privacy/index.html'),
 PosixPath('/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/Agents/docs.llamaindex.ai/en/latest/understanding/loading/llamahub/index.html')]

In [2]:

from llama_index.core import Document
from llama_index.readers.file import UnstructuredReader

reader = UnstructuredReader()

# TODO: set to higher value if you want more docs
doc_limit = 10

docs = []
for idx, f in enumerate(all_html_files):
    if idx > doc_limit:
        break
    print(f"Idx {idx}/{len(all_html_files)}")
    loaded_docs = reader.load_data(file=f, split_documents=True)
    # Hardcoded Index. Everything before this is ToC for all pages
    start_idx = 72
    loaded_doc = Document(
        text="\n\n".join([d.get_content() for d in loaded_docs[72:]]),
        metadata={"path": str(f)},
    )
    print(loaded_doc.metadata["path"])
    docs.append(loaded_doc)

Idx 0/1197
/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/Agents/docs.llamaindex.ai/en/latest/index.html
Idx 1/1197
/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/Agents/docs.llamaindex.ai/en/latest/understanding/index.html
Idx 2/1197
/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/Agents/docs.llamaindex.ai/en/latest/understanding/using_llms/using_llms/index.html
Idx 3/1197
/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/Agents/docs.llamaindex.ai/en/latest/understanding/using_llms/privacy/index.html
Idx 4/1197
/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/Agents/docs.llamaindex.ai/en/latest/understanding/loading/llamahub/index.html
Idx 5/1197
/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/Agents/docs.llamaindex.ai/en/latest/understanding/loading/loading/index.html
Idx 6/1197
/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/Agents/docs.llamaindex.ai/en/latest/understanding/evaluating/evaluating/index.html
Idx 7/1197
/Use

In [4]:
for doc in docs:
    print(doc.metadata["path"])
    print(doc.get_content())
    print("\n\n")

/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/Agents/docs.llamaindex.ai/en/latest/index.html




/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/Agents/docs.llamaindex.ai/en/latest/understanding/index.html




/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/Agents/docs.llamaindex.ai/en/latest/understanding/using_llms/using_llms/index.html
For example, if you have Ollama installed and running:

from

llama_index.llms.ollama

import

Ollama

from

llama_index.core

import

Settings

Settings

llm

Ollama

model

"llama2"

request_timeout

60.0

See the custom LLM's How-To for more details.

Prompts#

By default LlamaIndex comes with a great set of built-in, battle-tested prompts that handle the tricky work of getting a specific LLM to correctly handle and format data. This is one of the biggest benefits of using LlamaIndex. If you want to, you can customize the prompts



/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/Agents/docs.llamaindex.ai/en/la

#### Define Global LLM + Embeddings

In [4]:
import nest_asyncio
nest_asyncio.apply()

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

## 다중 문서 에이전트 구축
이 섹션에서는 다중 문서 에이전트를 구성하는 방법을 보여줍니다. 

먼저 각 문서에 대한 문서 에이전트를 구축한 다음 개체 인덱스를 사용하여 최상위 상위 에이전트를 정의합니다.




### 각 문서에 대한 문서 에이전트 구축
이 섹션에서는 각 문서에 대한 "문서 에이전트"를 정의합니다.

우리는 각 문서에 대해 벡터 인덱스(의미 검색용)와 요약 인덱스(요약용)를 모두 정의합니다. 그런 다음 두 개의 쿼리 엔진은 OpenAI 함수 호출 에이전트에 전달되는 도구로 변환됩니다.

이 문서 에이전트는 주어진 문서 내에서 의미 검색 또는 요약을 수행하도록 동적으로 선택할 수 있습니다.

우리는 각 도시마다 별도의 문서 에이전트를 만듭니다.

In [5]:
llm = OpenAI(model="gpt-3.5-turbo")

In [6]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.node_parser import SentenceSplitter
import os
from tqdm.notebook import tqdm
import pickle


async def build_agent_per_doc(nodes, file_base):
    print(file_base)

    vi_out_path = f"./data/llamaindex_docs/{file_base}"
    summary_out_path = f"./data/llamaindex_docs/{file_base}_summary.pkl"
    if not os.path.exists(vi_out_path):
        Path("./data/llamaindex_docs/").mkdir(parents=True, exist_ok=True)
        # build vector index
        vector_index = VectorStoreIndex(nodes)
        vector_index.storage_context.persist(persist_dir=vi_out_path)
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=vi_out_path),
        )

    # build summary index
    summary_index = SummaryIndex(nodes)

    # define query engines
    vector_query_engine = vector_index.as_query_engine(llm=llm)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize", llm=llm
    )

    # extract a summary
    if not os.path.exists(summary_out_path):
        Path(summary_out_path).parent.mkdir(parents=True, exist_ok=True)
        summary = str(
            await summary_query_engine.aquery(
                "Extract a concise 1-2 line summary of this document"
            )
        )
        pickle.dump(summary, open(summary_out_path, "wb"))
    else:
        summary = pickle.load(open(summary_out_path, "rb"))

    # define tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name=f"vector_tool_{file_base}",
                description=f"Useful for questions related to specific facts",
            ),
        ),
        QueryEngineTool(
            query_engine=summary_query_engine,
            metadata=ToolMetadata(
                name=f"summary_tool_{file_base}",
                description=f"Useful for summarization questions",
            ),
        ),
    ]

    # build agent
    function_llm = OpenAI(model="gpt-4")
    agent = OpenAIAgent.from_tools(
        query_engine_tools,
        llm=function_llm,
        verbose=True,
        system_prompt=f"""\
You are a specialized agent designed to answer queries about the `{file_base}.html` part of the LlamaIndex docs.
You must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.\
""",
    )

    return agent, summary


async def build_agents(docs):
    node_parser = SentenceSplitter()

    # Build agents dictionary
    agents_dict = {}
    extra_info_dict = {}

    # # this is for the baseline
    # all_nodes = []

    for idx, doc in enumerate(tqdm(docs)):
    # for idx, doc in enumerate(docs):
        nodes = node_parser.get_nodes_from_documents([doc])
        # all_nodes.extend(nodes)

        # ID will be base + parent
        file_path = Path(doc.metadata["path"])
        file_base = str(file_path.parent.stem) + "_" + str(file_path.stem)
        agent, summary = await build_agent_per_doc(nodes, file_base)

        agents_dict[file_base] = agent
        extra_info_dict[file_base] = {"summary": summary, "nodes": nodes}

    return agents_dict, extra_info_dict

In [7]:
agents_dict, extra_info_dict = await build_agents(docs)

  0%|          | 0/201 [00:00<?, ?it/s]

latest_index
understanding_index
using_llms_index
privacy_index
llamahub_index
loading_index
evaluating_index
cost_analysis_index
usage_pattern_index
storing_index
tracing_and_debugging_index
indexing_index
querying_index
putting_it_all_together_index
q_and_a_index
terms_definitions_tutorial_index
agents_index
structured_data_index
Airbyte_demo_index
building_a_chatbot_index
graphs_index
apps_index
fullstack_with_delphic_index
fullstack_app_guide_index
api_reference_index
retrievers_index
tree_index
auto_merging_index
knowledge_graph_index
videodb_index
query_fusion_index
recursive_index
bm25_index
pathway_index
you_index
keyword_index
vector_index
transform_index
summary_index
router_index
sql_index
ingestion_index
indices_index
tree_index
knowledge_graph_index
llama_cloud_index
google_index
vectara_index
colbert_index
document_summary_index
zilliz_index
keyword_index
vector_index
summary_index
tools_index
multion_index
waii_index
neo4j_index
database_index
metaphor_index
wikipedia_in

## 리트리버 지원 OpenAI 에이전트 빌드
우리는 다양한 문서 에이전트를 조정하여 모든 사용자 쿼리에 응답할 수 있는 최상위 에이전트를 구축합니다.

이는 RetrieverOpenAIAgent도구를 사용하기 전에 도구 검색을 수행합니다(모든 도구를 프롬프트에 넣으려고 시도하는 기본 에이전트와는 다름).

V0의 개선 사항 : V0의 "기본" 버전과 비교하여 다음과 같은 개선 사항을 적용했습니다.
- 순위 재지정 기능 추가: Cohere reranker를 사용하여 후보 문서 세트를 더 효과적으로 필터링합니다.
- 쿼리 계획 도구 추가: 검색된 도구 집합을 기반으로 동적으로 생성되는 명시적 쿼리 계획 도구를 추가합니다.

In [8]:
# define tool for each document agent
all_tools = []
for file_base, agent in agents_dict.items():
    summary = extra_info_dict[file_base]["summary"]
    doc_tool = QueryEngineTool(
        query_engine=agent,
        metadata=ToolMetadata(
            name=f"tool_{file_base}",
            description=summary,
        ),
    )
    all_tools.append(doc_tool)

In [9]:
print(all_tools[10].metadata)
print(len(all_tools))

ToolMetadata(description='This document provides guidance on tracing and debugging processes for effective troubleshooting in software development.', name='tool_tracing_and_debugging_index', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>)
182


In [10]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import (
    ObjectIndex,
    SimpleToolNodeMapping,
    ObjectRetriever,
)
from llama_index.core.retrievers import BaseRetriever
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4-0613")

tool_mapping = SimpleToolNodeMapping.from_objects(all_tools)
obj_index = ObjectIndex.from_objects(
    all_tools,
    tool_mapping,
    VectorStoreIndex,
)
vector_node_retriever = obj_index.as_node_retriever(similarity_top_k=10)


# define a custom retriever with reranking
class CustomRetriever(BaseRetriever):
    def __init__(self, vector_retriever, postprocessor=None):
        self._vector_retriever = vector_retriever
        self._postprocessor = postprocessor or CohereRerank(top_n=5)
        # self._postprocessor = postprocessor
        super().__init__()

    def _retrieve(self, query_bundle):
        retrieved_nodes = self._vector_retriever.retrieve(query_bundle)
        filtered_nodes = self._postprocessor.postprocess_nodes(
            retrieved_nodes, query_bundle=query_bundle
        )

        return filtered_nodes


# define a custom object retriever that adds in a query planning tool
class CustomObjectRetriever(ObjectRetriever):
    def __init__(self, retriever, object_node_mapping, all_tools, llm=None):
        self._retriever = retriever
        self._object_node_mapping = object_node_mapping
        self._llm = llm or OpenAI("gpt-4-0613")

    def retrieve(self, query_bundle):
        nodes = self._retriever.retrieve(query_bundle)
        tools = [self._object_node_mapping.from_node(n.node) for n in nodes]

        sub_question_engine = SubQuestionQueryEngine.from_defaults(
            query_engine_tools=tools, llm=self._llm
        )
        sub_question_description = f"""\
Useful for any queries that involve comparing multiple documents. ALWAYS use this tool for comparison queries - make sure to call this \
tool with the original query. Do NOT use the other tools for any queries involving multiple documents.
"""
        sub_question_tool = QueryEngineTool(
            query_engine=sub_question_engine,
            metadata=ToolMetadata(
                name="compare_tool", description=sub_question_description
            ),
        )

        return tools + [sub_question_tool]

In [11]:
from llama_index.core.postprocessor import SimilarityPostprocessor

# CohereRerank를 대신해서...
# postprocessor = SimilarityPostprocessor(similarity_cutoff=0.4)

custom_node_retriever = CustomRetriever(vector_node_retriever)

# wrap it with ObjectRetriever to return objects
custom_obj_retriever = CustomObjectRetriever(
    custom_node_retriever, tool_mapping, all_tools, llm=llm
)

In [12]:
llm.model

'gpt-4-0613'

In [13]:
tmps = custom_obj_retriever.retrieve("how can use your data to generate questions to evaluate on?")
print(len(tmps))

6


In [14]:
from llama_index.agent.openai_legacy import FnRetrieverOpenAIAgent
from llama_index.core.agent import ReActAgent

top_agent = FnRetrieverOpenAIAgent.from_retriever(
    custom_obj_retriever,
    system_prompt=""" \
You are an agent designed to answer queries about the documentation.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    llm=llm,
    verbose=True,
)

# top_agent = ReActAgent.from_tools(
#     tool_retriever=custom_obj_retriever,
#     system_prompt=""" \
# You are an agent designed to answer queries about the documentation.
# Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

# """,
#     llm=llm,
#     verbose=True,
# )

#### 도구함수 정의

In [15]:
def pretty_source_nodes(response):
    for node in response.source_nodes:
        print(node)

In [23]:
def save_llm_result(question, base_txt, agent_txt):
    # 파일을 추가 모드('a')로 열기
    with open('평가:Llama-index Document.md', 'a') as file:
        # 파일에 텍스트 추가
        file.write((
            f"\n\n------------------------------------------------------------------------------------------------------------------------------------------"
            f"\n\n### Q. {question}"
            f"\n##### Base:"
            f"\n{base_txt}"
            f"\n\n##### Agent:"
            f"\n{agent_txt}"
        ))


#### 벡터 저장소 인덱스 정의
비교를 위해 모든 문서를 단일 벡터 인덱스 컬렉션으로 덤프하는 "순진한" RAG 파이프라인을 정의합니다.

top_k = 4로 설정했습니다.

In [17]:
all_nodes = [
    n for extra_info in extra_info_dict.values() for n in extra_info["nodes"]
]

base_index = VectorStoreIndex(all_nodes)
base_query_engine = base_index.as_query_engine(similarity_top_k=4)

## 예제 쿼리 실행

단일 문서에 대한 QA/요약부터 여러 문서에 대한 QA/요약에 이르기까지 몇 가지 예시 쿼리를 실행해 보겠습니다.

In [24]:
questions = [
    # "Multi-Document Agents의 용도는 무엇이고 어떠한 usecase가 있는지 알려줘",
    # "Multi-Document Agents 사용에서 CohereRerank가 하는 역할이 무엇이고 이것이 미치는 영향이 큰가?",
    # "Llama-index에서 \"Response Synthesizer\"가 무엇이고 이것이 하는 역할과 Usecase를 알려줘",
    # "IndexNode가 무엇이고 이것이 하는 역할과 Usecase를 알려줘",
    # "ColbertRerank vs CohereRerank 두개를 비교했을때 장단점을 비교해주고 특히 RAG의 성능에 미치는 영향에 대해서 알려줘",
    # "FnRetrieverOpenAIAgent와 ReActAgent에 대해서 알려주고 이 두개의 특징을 비교해서 알려줘",
    "Llama-index에서 SentenceTransformerRerank에 대해서 설명해주고 CohereRerank와 어떤 차이점이 있는지 알려줘",
    "SentenceTransformerRerank의 Usecase에 대해서 알려줘"
]

In [25]:
for question in questions:
    base_response = base_query_engine.query(question)
    agent_response = top_agent.query(question)
    save_llm_result(question, str(base_response), str(agent_response))
    pretty_source_nodes(agent_response)
    print(f"\n\n\n")

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: compare_tool with args: {
  "input": "Llama-index에서 SentenceTransformerRerank에 대해서 설명해주고 CohereRerank와 어떤 차이점이 있는지 알려줘"
}
Generated 6 sub questions.
[tool_sentence_optimizer_index] Q: Llama-index에서 SentenceTransformerRerank에 대한 최적화 정보는 무엇인가요?
Added user message to memory: Llama-index에서 SentenceTransformerRerank에 대한 최적화 정보는 무엇인가요?
[tool_llm_rerank_index] Q: Llama-index에서 SentenceTransformerRerank의 후처리 단계는 어떻게 진행되나요?
Added user message to memory: Llama-index에서 SentenceTransformerRerank의 후처리 단계는 어떻게 진행되나요?
[tool_transform_index] Q: Llama-index에서 SentenceTransformerRerank를 변환하는 방법은 무엇인가요?
Added user message to memory: Llama-index에서 SentenceTransformerRerank를 변환하는 방법은 무엇인가요?
[tool_cohere_index] Q: Llama-index에서 CohereRerank의 임베딩 일관성 정보는 무엇인가요?
Added user message to memory: Llama-index에서 CohereRerank의 임베딩 일관성 정보는 무엇인가요?
[tool_rankgpt_rerank_index] Q: Llama-index에서 CohereRerank의 후처리 단계는 어떻게 진행되나요?
Added user message t